In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

calc_emergence =False
debris_only=False

min_dhdt_perc = 75

debug=False
extra_layers=True

csv_ending = '_mb_bins.csv'
# outdir_csv = debris_prms.mb_binned_fp
outdir_csv = ('/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/' + 
              '../output/mb_bins_all/csv/')
outdir_csv = outdir_csv + debris_prms.roi + '/'
if not os.path.exists(outdir_csv):
    os.makedirs(outdir_csv)
    
# dhdt_vel_fns_fp = debris_prms.dhdt_vel_fns_fp
dhdt_vel_fns_fp = ('/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/' + 
                   '../output/dhdt_vel_fns_all/')
if not os.path.exists(dhdt_vel_fns_fp):
    os.makedirs(dhdt_vel_fns_fp)


In [3]:
if debris_only:
    # Debris cover extent shapefile with statistics
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])

    # Subset by percent debris-covered or debris-covered area
    dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                            (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                            & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
    dc_shp_subset.reset_index(inplace=True, drop=True)
    dc_shp_subset['CenLon_360'] = dc_shp_subset['CenLon']
    dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'] = (
        360 + dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'])

    rgiid_list = [x.split('-')[1] for x in dc_shp_subset['RGIId'].values]
    main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgiid_list)

else:
    # All glaciers
    main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgi_regionsO1=debris_prms.roi_rgidict[debris_prms.roi],
                                                              rgi_regionsO2='all', rgi_glac_number='all')
    
    # Debris cover extent shapefile with statistics
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])
    dc_shp.reset_index(inplace=True, drop=True)

# dc_shp
main_glac_rgi_subset

All glaciers within region(s) 5 are included in this model run.
This study is focusing on 20261 glaciers in region [5]


,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,0,RGI60-05.00001,-53.2909,66.0765,5,1,0.168,689,995,868,29.0,25,406,0,0,9,20000814,1,05.00001,5.00001
1,1,RGI60-05.00002,-52.0705,65.3617,5,1,0.118,602,704,643,14.0,49,283,0,0,9,20000823,2,05.00002,5.00002
2,2,RGI60-05.00003,-52.0017,65.5791,5,1,0.059,646,697,683,10.1,28,365,0,0,9,20000823,3,05.00003,5.00003
3,3,RGI60-05.00004,-51.7992,65.1590,5,1,0.188,635,775,735,11.8,30,392,0,0,9,20000823,4,05.00004,5.00004
4,4,RGI60-05.00005,-51.7228,65.1786,5,1,0.086,711,769,747,13.1,1,218,0,0,9,20000823,5,05.00005,5.00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20256,20256,RGI60-05.20275,-29.7831,70.2707,5,1,1.008,1226,1560,1502,12.6,21,1093,1,0,9,20010921,20275,05.20275,5.20275
20257,20257,RGI60-05.20277,-29.7908,70.2829,5,1,0.071,1238,1282,1256,7.7,70,175,0,0,9,20010921,20277,05.20277,5.20277
20258,20258,RGI60-05.20278,-28.9625,69.9847,5,1,1.459,955,2066,1430,42.8,359,1697,1,0,9,20010712,20278,05.20278,5.20278


In [4]:
dhdt_vel_fns_fn = debris_prms.dhdt_vel_fns_fn.replace('XXXX',debris_prms.roi)
if os.path.exists(dhdt_vel_fns_fp + dhdt_vel_fns_fn):
    dhdt_vel_fns_df = pd.read_csv(dhdt_vel_fns_fp + dhdt_vel_fns_fn)
else:
    dhdt_vel_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_subset.shape[0], 3)),
                                   columns=['RGIId', 'dhdt_fullfn', 'vel_fullfn'])
    dhdt_vel_fns_df['RGIId'] = main_glac_rgi_subset['RGIId']

In [5]:
# np.where(main_glac_rgi_subset.rgino_str == '11.03005')

In [6]:
# skip existing
rgiid_existing = []
for i in os.listdir(outdir_csv):
    if i.endswith('.csv'):
        rgiid_existing.append(i.split('_')[0])
print('existing:', len(rgiid_existing))

existing: 0


In [8]:
# ===== PROCESS EACH GLACIER =====
dc_rgiids = dc_shp.RGIId.tolist()
for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
# for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values[15066:]):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[120]]): # Miage
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[2307]]): # Ngozumpa

    glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
    rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
    region = glac_str.split('.')[0]

    if int(region) < 10:
        glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
    else:
        glac_str_noleadzero = glac_str
        
    if glac_str_noleadzero not in rgiid_existing:

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        
        if os.path.exists(thick_dir + thick_fn):
        
            gf = create_glacfeat(thick_dir, thick_fn)

            if rgiid in dc_rgiids:
                # Debris shape layer processing
                dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                                  str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
                if os.path.exists(dc_shp_proj_fn) == False:
                    dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                    dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                    dc_shp_single = dc_shp_single.reset_index()
                    dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                    dc_shp_proj.to_file(dc_shp_proj_fn)
                dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
                dc_shp_lyr = dc_shp_ds.GetLayer()
            else:
                dc_shp_lyr = None

            # ==== CHECK IF TIF HAS DHDT DATA OVER THE GLACIER =====
            mb_fullfns = []
            find_mb = True
            dhdt_fn_wglacier = None
            for mb_fp in debris_prms.mb_fp_list_roi[debris_prms.roi]:
                if find_mb:
                    for i in os.listdir(mb_fp):
                        if i.endswith('.tif'):
                            mb_fullfns.append(mb_fp + i)
                    tif_count = 0
                    while find_mb and tif_count < len(mb_fullfns):
                        dhdt_fn = mb_fullfns[tif_count]
                        if debug:
                            print(tif_count, dhdt_fn.split('/')[-1])

                        # Add the filenames
                        fn_dict = OrderedDict()
                        # DEM
                        z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
                        z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
                        fn_dict['z1'] = z1_fp + z1_fn
                        # Ice thickness
                        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
                        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
                        fn_dict['ice_thick'] = thick_dir + thick_fn
                        # dh/dt
                        fn_dict['dhdt'] = dhdt_fn
                        # ===== PROCESS THE DATA =====
                        #Expand extent to include buffered region around glacier polygon
                        warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
                        #Warp everything to common res/extent/proj
                        z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
                        z1_res = np.min([z1_gt[1], -z1_gt[5]])
                        ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                           t_srs=gf.aea_srs, verbose=False, r='cubic')
                        ds_dict = dict(zip(fn_dict.keys(), ds_list))
                        gf.ds_dict = ds_dict

                        if 'z1' in ds_dict:
                            #This is False over glacier polygon surface, True elsewhere - can be applied directly
                            glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                            glac_geom_mask_copy = glac_geom_mask.copy()
                            gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                            if rgiid in dc_rgiids:
                                # Debris cover
                                dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                                gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                            if 'dhdt' in ds_dict:
                                gf.dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                gf.dhdt.mask = np.ma.mask_or(
                                    glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.dhdt.data, 
                                                                                     np.isnan(gf.dhdt.data))))
                                
                                if rgiid in dc_rgiids:
                                    gf.dc_dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=gf.dc_mask)
                                    gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)
                                # Count dhdt pixels
                                dhdt_pixels = len(gf.dhdt.nonzero()[0])                              
                                
                                if dhdt_pixels / gf.z1.count() * 100 > min_dhdt_perc:
                                    dhdt_fn_wglacier = dhdt_fn
                                    find_mb = False
                                    if debug:
                                        print('\n# z1 pixels:', gf.z1.count())
                                        print('# dhdt_pixels:', dhdt_pixels)
                                        var_full2plot = gf.dhdt.copy()
                                        clim = malib.calcperc(var_full2plot, (2,98))
                                        plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', 
                                                   close_fig=False)
                        # Loop over layers        
                        tif_count += 1

            # ==== CHECK IF VELOCITY DATA OVER THE GLACIER =====
            vel_fullfns = []
            find_vel = True
            vx_fn_wglacier = None
            if find_vel and dhdt_fn_wglacier is not None:
                vx_fns = debris_prms.vx_dir_dict_list[debris_prms.roi]
                tif_count = 0
                while find_vel and tif_count < len(vx_fns):
                    vx_fn = vx_fns[tif_count]

                    if debug:
                        print(tif_count, vx_fn.split('/')[-1])

                    # Add the filenames
                    # Velocity
                    fn_dict['vx'] = vx_fn
                    fn_dict['vy'] = vx_fn.replace('_vx', '_vy')
                    # ===== PROCESS THE DATA =====
                    ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                       t_srs=gf.aea_srs, verbose=False, r='cubic')
                    ds_dict = dict(zip(fn_dict.keys(), ds_list))
                    gf.ds_dict = ds_dict
                    if 'vx' in ds_dict and 'vy' in ds_dict:
                        #Load surface velocity maps
                        gf.vx = np.ma.array(iolib.ds_getma(ds_dict['vx']), mask=glac_geom_mask)
                        gf.vy = np.ma.array(iolib.ds_getma(ds_dict['vy']), mask=glac_geom_mask)
                        gf.vm = np.ma.sqrt(gf.vx**2 + gf.vy**2)
                        # Count velocity pixels
                        vel_pixels = len(gf.vm.nonzero()[0])
                        if debug:
                                print('\n# z1 pixels:', gf.z1.count())
                                print('# vel_pixels:', vel_pixels)
                                var_full2plot = gf.vm.copy()
                                clim = malib.calcperc(var_full2plot, (2,98))
                                plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vm (m/yr)', 
                                           close_fig=False)
                        if vel_pixels / gf.z1.count() * 100 > min_dhdt_perc:
                            vx_fn_wglacier = vx_fn
                            find_vel = False

                    # Loop over layers        
                    tif_count += 1


            # ===== Add layers =====
            if dhdt_fn_wglacier is not None and vx_fn_wglacier is not None:
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=True, dhdt_fn=dhdt_fn_wglacier, gf_add_vel=True, 
                              vx_fn=vx_fn_wglacier, gf_add_ts=False, gf_add_slope_aspect=True, gf_add_ts_info=False, 
                              calc_emergence=calc_emergence, debug_emergence=False)
                # Save dhdt and vel filenames
                dhdt_vel_fns_df.loc[glac_idx,:] = [rgiid, dhdt_fn_wglacier, vx_fn_wglacier]

                # ===== PLOTS =====
#                 plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    # Elevation change
                    var_full2plot = gf.dhdt.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', close_fig=False)
                    # Velocity
                    var_full2plot = gf.vm.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vel (m/yr)', close_fig=False)
                    # Emergence velocity
                    if gf.emvel is not None:
                        var_full2plot = gf.emvel.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' emvel'], 'inferno', 'emvel (m/yr)', close_fig=False)
                    # Surface temperature
                    if gf.ts is not None:
                        var_full2plot = gf.ts.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)

                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:7] + csv_ending)
                else:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)
            
            # Elevation bins for glaciers with only dhdt data
            elif dhdt_fn_wglacier is not None and vx_fn_wglacier is None:
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=True, dhdt_fn=dhdt_fn_wglacier, gf_add_vel=False, 
                              gf_add_ts=False, gf_add_slope_aspect=True, gf_add_ts_info=False, 
                              calc_emergence=calc_emergence, debug_emergence=False)
                # Save dhdt and vel filenames
                dhdt_vel_fns_df.loc[glac_idx,:] = [rgiid, dhdt_fn_wglacier, 0]

                # ===== PLOTS =====
#                 plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    # Elevation change
                    var_full2plot = gf.dhdt.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', close_fig=False)
                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:7] + csv_ending)
                else:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)
            
            # Elevation bins for glaciers without any data
            else:
                gf.dhdt, gf.dc_dhdt = None, None
                gf.vx, gf.vy, gf.vm = None, None, None
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=False, gf_add_vel=False, gf_add_ts=False, 
                              gf_add_slope_aspect=True, gf_add_ts_info=False, 
                              calc_emergence=calc_emergence, debug_emergence=False,
                             verbose=False)
                # ===== PLOTS =====
#                 plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    
                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                outdir_csv_nodhdt = outdir_csv + 'no_dhdt/'
                csv_ending_nodhdt = '_bins.csv'
                if not os.path.exists(outdir_csv_nodhdt):
                    os.makedirs(outdir_csv_nodhdt)
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv_nodhdt, gf.feat_fn[0:7] + csv_ending_nodhdt)
                else:
                    outbins_fullfn = os.path.join(outdir_csv_nodhdt, gf.feat_fn[0:8] + csv_ending_nodhdt)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)


# Save updated filenames
dhdt_vel_fns_df.to_csv(dhdt_vel_fns_fp + dhdt_vel_fns_fn, index=False)

0 15066 RGI60-05.15072
1 15067 RGI60-05.15073
2 15068 RGI60-05.15074
3 15069 RGI60-05.15075
4 15070 RGI60-05.15076
5 15071 RGI60-05.15077
6 15072 RGI60-05.15078
7 15073 RGI60-05.15079
8 15074 RGI60-05.15080
9 15075 RGI60-05.15081
10 15076 RGI60-05.15082
11 15077 RGI60-05.15083
12 15078 RGI60-05.15084
13 15079 RGI60-05.15085
14 15080 RGI60-05.15086
15 15081 RGI60-05.15087
16 15082 RGI60-05.15088
17 15083 RGI60-05.15089
18 15084 RGI60-05.15090
19 15085 RGI60-05.15091
20 15086 RGI60-05.15092
21 15087 RGI60-05.15093
22 15088 RGI60-05.15094
23 15089 RGI60-05.15095
24 15090 RGI60-05.15096
25 15091 RGI60-05.15097
26 15092 RGI60-05.15098
27 15093 RGI60-05.15099
28 15094 RGI60-05.15100
29 15095 RGI60-05.15101
30 15096 RGI60-05.15102
31 15097 RGI60-05.15103
32 15098 RGI60-05.15104
33 15099 RGI60-05.15105
34 15100 RGI60-05.15106
35 15101 RGI60-05.15107
36 15102 RGI60-05.15108
37 15103 RGI60-05.15109
38 15104 RGI60-05.15110
39 15105 RGI60-05.15111
40 15106 RGI60-05.15112
41 15107 RGI60-05.15113
42

333 15399 RGI60-05.15405
334 15400 RGI60-05.15406
335 15401 RGI60-05.15407
336 15402 RGI60-05.15408
337 15403 RGI60-05.15409
338 15404 RGI60-05.15410
339 15405 RGI60-05.15411
340 15406 RGI60-05.15412
341 15407 RGI60-05.15413
342 15408 RGI60-05.15414
343 15409 RGI60-05.15415
344 15410 RGI60-05.15416
345 15411 RGI60-05.15417
346 15412 RGI60-05.15418
347 15413 RGI60-05.15419
348 15414 RGI60-05.15420
349 15415 RGI60-05.15421
350 15416 RGI60-05.15422
351 15417 RGI60-05.15423
352 15418 RGI60-05.15424
353 15419 RGI60-05.15425
354 15420 RGI60-05.15426
355 15421 RGI60-05.15427
356 15422 RGI60-05.15428
357 15423 RGI60-05.15429
358 15424 RGI60-05.15430
359 15425 RGI60-05.15431
360 15426 RGI60-05.15432
361 15427 RGI60-05.15433
362 15428 RGI60-05.15434
363 15429 RGI60-05.15435
364 15430 RGI60-05.15436
365 15431 RGI60-05.15437
366 15432 RGI60-05.15438
367 15433 RGI60-05.15439
368 15434 RGI60-05.15440
369 15435 RGI60-05.15441
370 15436 RGI60-05.15442
371 15437 RGI60-05.15443
372 15438 RGI60-05.15444


661 15727 RGI60-05.15733
662 15728 RGI60-05.15734
663 15729 RGI60-05.15735
664 15730 RGI60-05.15736
665 15731 RGI60-05.15737
666 15732 RGI60-05.15738
667 15733 RGI60-05.15739
668 15734 RGI60-05.15740
669 15735 RGI60-05.15741
670 15736 RGI60-05.15742
671 15737 RGI60-05.15743
672 15738 RGI60-05.15744
673 15739 RGI60-05.15745
674 15740 RGI60-05.15746
675 15741 RGI60-05.15747
676 15742 RGI60-05.15748
677 15743 RGI60-05.15749
678 15744 RGI60-05.15750
679 15745 RGI60-05.15751
680 15746 RGI60-05.15752
681 15747 RGI60-05.15753
682 15748 RGI60-05.15754
683 15749 RGI60-05.15755
684 15750 RGI60-05.15756
685 15751 RGI60-05.15757
686 15752 RGI60-05.15758
687 15753 RGI60-05.15759
688 15754 RGI60-05.15760
689 15755 RGI60-05.15761
690 15756 RGI60-05.15762
691 15757 RGI60-05.15763
692 15758 RGI60-05.15764
693 15759 RGI60-05.15765
694 15760 RGI60-05.15766
695 15761 RGI60-05.15767
696 15762 RGI60-05.15768
697 15763 RGI60-05.15769
698 15764 RGI60-05.15770
699 15765 RGI60-05.15771
700 15766 RGI60-05.15772


989 16055 RGI60-05.16061
990 16056 RGI60-05.16062
991 16057 RGI60-05.16063
992 16058 RGI60-05.16064
993 16059 RGI60-05.16065
994 16060 RGI60-05.16066
995 16061 RGI60-05.16067
996 16062 RGI60-05.16068
997 16063 RGI60-05.16069
998 16064 RGI60-05.16070
999 16065 RGI60-05.16071
1000 16066 RGI60-05.16072
1001 16067 RGI60-05.16073
1002 16068 RGI60-05.16074
1003 16069 RGI60-05.16075
1004 16070 RGI60-05.16076
1005 16071 RGI60-05.16077
1006 16072 RGI60-05.16078
1007 16073 RGI60-05.16079
1008 16074 RGI60-05.16080
1009 16075 RGI60-05.16081
1010 16076 RGI60-05.16082
1011 16077 RGI60-05.16083
1012 16078 RGI60-05.16084
1013 16079 RGI60-05.16085
1014 16080 RGI60-05.16086
1015 16081 RGI60-05.16087
1016 16082 RGI60-05.16088
1017 16083 RGI60-05.16089
1018 16084 RGI60-05.16090
1019 16085 RGI60-05.16091
1020 16086 RGI60-05.16092
1021 16087 RGI60-05.16093
1022 16088 RGI60-05.16094
1023 16089 RGI60-05.16095
1024 16090 RGI60-05.16096
1025 16091 RGI60-05.16097
1026 16092 RGI60-05.16098
1027 16093 RGI60-05.160

1305 16371 RGI60-05.16377
1306 16372 RGI60-05.16378
1307 16373 RGI60-05.16379
1308 16374 RGI60-05.16380
1309 16375 RGI60-05.16381
1310 16376 RGI60-05.16382
1311 16377 RGI60-05.16383
1312 16378 RGI60-05.16384
1313 16379 RGI60-05.16385
1314 16380 RGI60-05.16386
1315 16381 RGI60-05.16387
1316 16382 RGI60-05.16388
1317 16383 RGI60-05.16389
1318 16384 RGI60-05.16390
1319 16385 RGI60-05.16391
1320 16386 RGI60-05.16392
1321 16387 RGI60-05.16393
1322 16388 RGI60-05.16394
1323 16389 RGI60-05.16395
1324 16390 RGI60-05.16396
1325 16391 RGI60-05.16397
1326 16392 RGI60-05.16398
1327 16393 RGI60-05.16399
1328 16394 RGI60-05.16400
1329 16395 RGI60-05.16401
1330 16396 RGI60-05.16402
1331 16397 RGI60-05.16403
1332 16398 RGI60-05.16404
1333 16399 RGI60-05.16405
1334 16400 RGI60-05.16406
1335 16401 RGI60-05.16407
1336 16402 RGI60-05.16408
1337 16403 RGI60-05.16409
1338 16404 RGI60-05.16410
1339 16405 RGI60-05.16411
1340 16406 RGI60-05.16412
1341 16407 RGI60-05.16413
1342 16408 RGI60-05.16414
1343 16409 R

...

3833 18899 RGI60-05.18905
3834 18900 RGI60-05.18906
3835 18901 RGI60-05.18907
3836 18902 RGI60-05.18908
3837 18903 RGI60-05.18909
3838 18904 RGI60-05.18910
3839 18905 RGI60-05.18911
3840 18906 RGI60-05.18912
3841 18907 RGI60-05.18913
3842 18908 RGI60-05.18914
3843 18909 RGI60-05.18915
3844 18910 RGI60-05.18916
3845 18911 RGI60-05.18917
3846 18912 RGI60-05.18918
3847 18913 RGI60-05.18919
3848 18914 RGI60-05.18920
3849 18915 RGI60-05.18921
3850 18916 RGI60-05.18922
3851 18917 RGI60-05.18923
3852 18918 RGI60-05.18924
3853 18919 RGI60-05.18925
3854 18920 RGI60-05.18926
3855 18921 RGI60-05.18927
3856 18922 RGI60-05.18928
3857 18923 RGI60-05.18929
3858 18924 RGI60-05.18930
3859 18925 RGI60-05.18931
3860 18926 RGI60-05.18932
3861 18927 RGI60-05.18933
3862 18928 RGI60-05.18934
3863 18929 RGI60-05.18935
3864 18930 RGI60-05.18936
3865 18931 RGI60-05.18937
3866 18932 RGI60-05.18938
3867 18933 RGI60-05.18939
3868 18934 RGI60-05.18940
3869 18935 RGI60-05.18941
3870 18936 RGI60-05.18942
3871 18937 R

4149 19215 RGI60-05.19221
4150 19216 RGI60-05.19222
4151 19217 RGI60-05.19223
4152 19218 RGI60-05.19224
4153 19219 RGI60-05.19225
4154 19220 RGI60-05.19226
4155 19221 RGI60-05.19227
4156 19222 RGI60-05.19228
4157 19223 RGI60-05.19229
4158 19224 RGI60-05.19230
4159 19225 RGI60-05.19231
4160 19226 RGI60-05.19232
4161 19227 RGI60-05.19233
4162 19228 RGI60-05.19234
4163 19229 RGI60-05.19235
4164 19230 RGI60-05.19236
4165 19231 RGI60-05.19237
4166 19232 RGI60-05.19238
4167 19233 RGI60-05.19239
4168 19234 RGI60-05.19240
4169 19235 RGI60-05.19241
4170 19236 RGI60-05.19242
4171 19237 RGI60-05.19243
4172 19238 RGI60-05.19244
4173 19239 RGI60-05.19245
4174 19240 RGI60-05.19246
4175 19241 RGI60-05.19247
4176 19242 RGI60-05.19248
4177 19243 RGI60-05.19249
4178 19244 RGI60-05.19250
4179 19245 RGI60-05.19251
4180 19246 RGI60-05.19252
4181 19247 RGI60-05.19253
4182 19248 RGI60-05.19254
4183 19249 RGI60-05.19255
4184 19250 RGI60-05.19256
4185 19251 RGI60-05.19257
4186 19252 RGI60-05.19258
4187 19253 R

4465 19531 RGI60-05.19537
4466 19532 RGI60-05.19538
4467 19533 RGI60-05.19539
4468 19534 RGI60-05.19540
4469 19535 RGI60-05.19541
4470 19536 RGI60-05.19542
4471 19537 RGI60-05.19543
4472 19538 RGI60-05.19544
4473 19539 RGI60-05.19545
4474 19540 RGI60-05.19546
4475 19541 RGI60-05.19547
4476 19542 RGI60-05.19548
4477 19543 RGI60-05.19549
4478 19544 RGI60-05.19550
4479 19545 RGI60-05.19551
4480 19546 RGI60-05.19552
4481 19547 RGI60-05.19553
4482 19548 RGI60-05.19554
4483 19549 RGI60-05.19555
4484 19550 RGI60-05.19556
4485 19551 RGI60-05.19557
4486 19552 RGI60-05.19558
4487 19553 RGI60-05.19559
4488 19554 RGI60-05.19560
4489 19555 RGI60-05.19561
4490 19556 RGI60-05.19562
4491 19557 RGI60-05.19563
4492 19558 RGI60-05.19564
4493 19559 RGI60-05.19565
4494 19560 RGI60-05.19566
4495 19561 RGI60-05.19567
4496 19562 RGI60-05.19568
4497 19563 RGI60-05.19569
4498 19564 RGI60-05.19570
4499 19565 RGI60-05.19571
4500 19566 RGI60-05.19572
4501 19567 RGI60-05.19573
4502 19568 RGI60-05.19574
4503 19569 R

4781 19847 RGI60-05.19853
4782 19848 RGI60-05.19854
4783 19849 RGI60-05.19855
4784 19850 RGI60-05.19856
4785 19851 RGI60-05.19857
4786 19852 RGI60-05.19858
4787 19853 RGI60-05.19859
4788 19854 RGI60-05.19860
4789 19855 RGI60-05.19861
4790 19856 RGI60-05.19862
4791 19857 RGI60-05.19863
4792 19858 RGI60-05.19864
4793 19859 RGI60-05.19865
4794 19860 RGI60-05.19866
4795 19861 RGI60-05.19867
4796 19862 RGI60-05.19868
4797 19863 RGI60-05.19869
4798 19864 RGI60-05.19870
4799 19865 RGI60-05.19871
4800 19866 RGI60-05.19872
4801 19867 RGI60-05.19873
4802 19868 RGI60-05.19874
4803 19869 RGI60-05.19875
4804 19870 RGI60-05.19876
4805 19871 RGI60-05.19877
4806 19872 RGI60-05.19878
4807 19873 RGI60-05.19879
4808 19874 RGI60-05.19880
4809 19875 RGI60-05.19881
4810 19876 RGI60-05.19882
4811 19877 RGI60-05.19883
4812 19878 RGI60-05.19884
4813 19879 RGI60-05.19885
4814 19880 RGI60-05.19886
4815 19881 RGI60-05.19887
4816 19882 RGI60-05.19888
4817 19883 RGI60-05.19889
4818 19884 RGI60-05.19890
4819 19885 R

5097 20163 RGI60-05.20178
5098 20164 RGI60-05.20179
5099 20165 RGI60-05.20180
5100 20166 RGI60-05.20181
5101 20167 RGI60-05.20182
5102 20168 RGI60-05.20183
5103 20169 RGI60-05.20184
5104 20170 RGI60-05.20185
5105 20171 RGI60-05.20186
5106 20172 RGI60-05.20187
5107 20173 RGI60-05.20188
5108 20174 RGI60-05.20189
5109 20175 RGI60-05.20190
5110 20176 RGI60-05.20191
5111 20177 RGI60-05.20192
5112 20178 RGI60-05.20193
5113 20179 RGI60-05.20194
5114 20180 RGI60-05.20195
5115 20181 RGI60-05.20196
5116 20182 RGI60-05.20197
5117 20183 RGI60-05.20198
5118 20184 RGI60-05.20199
5119 20185 RGI60-05.20200
5120 20186 RGI60-05.20201
5121 20187 RGI60-05.20202
5122 20188 RGI60-05.20203
5123 20189 RGI60-05.20204
5124 20190 RGI60-05.20205
5125 20191 RGI60-05.20206
5126 20192 RGI60-05.20207
5127 20193 RGI60-05.20208
5128 20194 RGI60-05.20209
5129 20195 RGI60-05.20210
5130 20196 RGI60-05.20211
5131 20197 RGI60-05.20212
5132 20198 RGI60-05.20213
5133 20199 RGI60-05.20214
5134 20200 RGI60-05.20215
5135 20201 R

In [ ]:
print('\n\nDONE!\n\n')

In [9]:
# skip existing
rgiid_existing = []
for i in os.listdir(outdir_csv):
    if i.endswith('.csv'):
        rgiid_existing.append(i.split('_')[0])
main_glac_rgi_processed = debris_prms.selectglaciersrgitable(rgiid_existing)
print('Processed ' + str(len(rgiid_existing)) + ' of ' + str(len(main_glac_rgi_subset)) + ' glaciers')
area_total = main_glac_rgi_subset.Area.sum()
area_processed = main_glac_rgi_processed.Area.sum()
print('  Area: ' + str(np.round(area_processed,1)) + ' km2 (' + str(np.round(area_processed / area_total * 100,1)) + 
      '%) of ' +  str(np.round(area_total,1)) + ' km2')

19212 glaciers in region 5 are included in this model run: ['00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042', '00043', '00044', '00045', '00046', '00047', '00048', '00049', '00050'] and more
This study is focusing on 19212 glaciers in region [5]
Processed 19212 of 20261 glaciers
  Area: 87892.2 km2 (67.6%) of 130071.2 km2


In [29]:
# # Isolate HMA regions
# # outdir_csv_raw = outdir_csv.replace('HMA/', '')
# if debris_prms.roi == 'HMA':
#     for region in ['13','14','15']:
#         outdir_csv = outdir_csv_raw + region + '/'
#         rgiid_existing = []
#         for i in os.listdir(outdir_csv):
#             if i.endswith('.csv'):
#                 rgiid_existing.append(i.split('_')[0])
#         main_glac_rgi_processed = debris_prms.selectglaciersrgitable(rgiid_existing)
#         main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgi_regionsO1=[int(region)],
#                                                                   rgi_regionsO2='all', rgi_glac_number='all')
#         area_total = main_glac_rgi_subset.Area.sum()
#         area_processed = main_glac_rgi_processed.Area.sum()
#         print('Processed ' + str(len(rgiid_existing)) + ' of ' + str(len(main_glac_rgi_subset)) + ' glaciers')
#         print('  Area: ' + str(np.round(area_processed,1)) + ' km2 (' + str(np.round(area_processed / area_total * 100,1)) + 
#               '%) of ' +  str(np.round(area_total,1)) + ' km2\n')

In [10]:
# ===== SHEAN ESTIMATE OF FLUX DIVERGENCE QUICKLY ======
#                 if gf.H is not None:
#                     #Compute flux
#                     gf.Q = gf.H * debris_prms.v_col_f * np.array([gf.vx, gf.vy])
#                     #Note: np.gradient returns derivatives relative to axis number, so (y, x) in this case
#                     #Want x-derivative of x component
#                     gf.divQ = np.gradient(gf.Q[0])[1] + np.gradient(gf.Q[1])[0]
# #                     gf.divQ = gf.H*(np.gradient(v_col_f*gf.vx)[1] + np.gradient(v_col_f*gf.vy)[0]) \
# #                             + v_col_f*gf.vx*(np.gradient(gf.H)[1]) + v_col_f*gf.vy*(np.gradient(gf.H)[0])
#                     #Should smooth divQ, better handling of data gaps

In [ ]:
# ===== OLD CHECK DEM FOR ERRORS AND REPLACE SCRIPT (no longer needed with OGGM processing) =====
#         #Create buffer around glacier polygon
#         glac_geom_buff = gf.glac_geom.Buffer(debris_prms.buff_dist)
#         #This is False over glacier polygon surface, True elsewhere - can be applied directly
#         glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])
        
#             # ds masks
#             ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#             dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
#             dems_mask = dem1.mask
#             if verbose:
#                 print('list of datasets:', len(ds_list_masked), fn_dict.values())

#             #Combine to identify ~1 km buffer around glacier polygon over static rock
#             static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#             static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)
        
        
#             # Check if DEM has huge errors or not - replace if necessary
#             if input.roi in ['01']:

#                 gf.z1_check = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
#                 if gf.z1_check.min() < 0:

#                     # Add backup DEM for regions with known poor quality (ex. Alaska)
#                     print('switching DEMs')
#                     fn_dict['z1_backup'] = input.z1_backup_dict[input.roi]
#                     # Warp everything to common res/extent/proj (a second time)
#                     ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, \
#                             extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
#                             r='cubic')
#                     ds_dict = dict(zip(fn_dict.keys(), ds_list))

#                     if verbose:
#                         print(ds_list)
#                         print(fn_dict.keys())

#                     # ds masks
#                     ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#                     dem1 = np.ma.masked_less_equal(ds_list_masked[-1], 0)
#                     dems_mask = dem1.mask
#                     if verbose:
#                         print('list of datasets:', len(ds_list_masked), fn_dict.values())

#                     #Combine to identify ~1 km buffer around glacier polygon over static rock
#                     static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#                     static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

#                     #This is False over glacier polygon surface, True elsewhere - can be applied directly
#                     glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1_backup'])
#                     gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=glac_geom_mask)
#                     #gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)

#                     # Debris cover
#                     dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
#                     gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=dc_mask)

In [7]:
rgi_shp = gpd.read_file('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/qgis_himat/rgi60_HMA.shp')
hma_fp = '/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/output/mb_bins_all/csv/HMA/'
rgiids = []
for i in os.listdir(hma_fp):
    if i.endswith('.csv'):
        rgiids.append('RGI60-' + i.split('_')[0])
rgiids = sorted(rgiids)

rgiids_all = list(rgi_shp.RGIId.values)
rgiids_missing = list(set(rgiids_all) - set(rgiids))

rgiids_missing_idx = []
for nglac, x in enumerate(rgi_shp.RGIId.values):
    if x in rgiids_missing:
        rgiids_missing_idx.append(nglac)
        
rgi_shp_missing = rgi_shp.loc[rgiids_missing_idx,:]
# rgi_shp_missing

output_fn = '/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/output/hma_missing.shp'
rgi_shp_missing.to_file(output_fn)